In [2]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from radstats import environment,models

In [3]:
### plotly template
template = go.layout.Template()
template.layout = {
    'paper_bgcolor':'rgba(0,0,0,0)',
    'plot_bgcolor':'rgba(0,0,0,0)',
    'xaxis':{'tickformat':'none','hoverformat':'.3e','showline':True,'mirror':True},
    'yaxis':{'tickformat':'.0e','hoverformat':'.3e','showline':True,'mirror':True},
    'width':400,
    'height':300,
    'font':{'family':'Times New Roman'}
}
template.data.scatter = [go.Scatter(
    marker = {'symbol':'circle-open','size':5},
    line = {'width':1}
)]
pio.templates.default = template
pio.renderers.default = 'notebook'

In [4]:
project = 'projects/new'
orbits = 1
creme_prefix = 'gto'

In [5]:
orbits = 1
creme_prefix = 'gto'

### read spenvis trajectory
env = environment.read_spenvis(f'{project}/spenvis/spenvis_sao.txt')[1]['DF']
env['L-shell'] = environment.read_spenvis(f'{project}/spenvis/spenvis_spp.txt')[1]['DF']['L (!8R!3!dE!n)']
env.index = pd.to_timedelta(env['MJD (days)']-int(env['MJD (days)'][0]),unit='days')
env.index = pd.Series(env.index,name='Time (s)').dt.seconds

### repeat orbits
env.index = pd.Series(env.index).diff().bfill()
env.loc[:,'Orbit'] = 0
for o in range(1,orbits):
    orbit = env.loc[env['Orbit'] == 0]
    orbit.loc[:,'Orbit'] = o
    env = pd.concat([env,orbit])
env.index = np.around(pd.Series(env.index).cumsum() - env.index[0],5)

### read creme files
l_bins = np.around(np.geomspace(0.8*env['L-shell'].min(),1.2*env['L-shell'].max(),10),2)
print('CREME bins',l_bins[:-1])
bin_width = np.diff(l_bins)
l_bins = l_bins[:-1] + np.diff(l_bins) / 2
print('L bins',l_bins)
print('Trajectory L min/max',[env['L-shell'].min(),env['L-shell'].max()])

effects = []
for i,l in enumerate(l_bins):
    file = open(f'{project}/creme/{creme_prefix}{i+1}.hup').read()
    N = len(file.split('REPORT')) - 1
    for n in range(N): effects.append(file.split('\n')[38+12*n].split(':')[1].split(' T ')[0].strip())
    file = open(f'{project}/creme/{creme_prefix}{i+1}.pup').read()
    N = len(file.split('REPORT')) - 1
    for n in range(N): effects.append(file.split('\n')[33+10*n].split(':')[1].split(' T ')[0].strip())
effects = list(set(effects))

rates = pd.DataFrame(index=pd.MultiIndex.from_product([l_bins,effects],names=['L-shell','effect']),
    columns=['dose','hup','pup'],dtype=float)

for i,l in enumerate(l_bins):
    ### get dose rate
    rates['dose'][l,] = float(open(f'{project}/creme/{creme_prefix}{i+1}.dse').read().split('\n')[2].split('=')[1].split('rad')[0])
    ### get heavy ion upset rate
    file = open(f'{project}/creme/{creme_prefix}{i+1}.hup').read()
    N = len(file.split('REPORT')) - 1
    for n in range(N):
        effect = file.split('\n')[38+12*n].split(':')[1].split(' T ')[0].strip()
        rates['hup'][l,effect] = float(file.split('\n')[48+12*n][12:32].strip())
    ### get proton upset rate
    file = open(f'{project}/creme/{creme_prefix}{i+1}.pup').read()
    N = len(file.split('REPORT')) - 1
    for n in range(N):
        effect = file.split('\n')[33+10*n].split(':')[1].split(' T ')[0].strip()
        rates['pup'][l,effect] = float(file.split('\n')[41+10*n][12:32].strip())
    
    rates['upset'] = rates['hup'].fillna(0) + rates['pup'].fillna(0)

### get SEE probs
spline = {}
for effect in effects: spline[effect] = interp1d(l_bins,np.log(rates['upset'][:,effect]),'cubic')
spline['DOSE (rad)'] = interp1d(l_bins,np.log(rates['dose'][:,effect]),'cubic')
R = pd.DataFrame(index=env.index,columns=['dose'] + effects)
R['DOSE (rad)'] = np.exp(spline['DOSE (rad)'](env['L-shell']))
for effect in effects: R[effect] = np.exp(spline[effect](env['L-shell']))

tid = list((R['DOSE (rad)'] * pd.Series(R.index).diff().bfill().values).cumsum() / 1000)
env['Dose Rate (rad/s)'] = R['DOSE (rad)']
env['Total Dose (rad)'] = tid

x = np.linspace(env['L-shell'].min(),env['L-shell'].max(),1000)
splines = pd.DataFrame(index=x)
for key in spline.keys():
    splines[key] = np.exp(spline[key](x))
cmap = {a:b for a,b in zip(effects + ['DOSE (rad)'],px.colors.qualitative.Plotly)}
fig = px.line(splines.drop(columns=['REG_SET']),log_y=True,color_discrete_map=cmap,labels={'value':'','index':'L-shell','variable':'Rates (/s)'})
fig.add_traces(list(px.scatter(rates['upset'].unstack(),color_discrete_map=cmap).update_traces(showlegend=False).select_traces()))
fig.add_trace(go.Scatter(x=l_bins, y=rates['dose'][:,effects[0]],name='DOSE (rad)',mode='markers',
    marker={'color':cmap['DOSE (rad)']},showlegend=False))
fig.update_traces(line={'width':1})
fig.write_image(f'{project}/figures/splines.png',scale=3)
fig.show()

### get TID probs (in RADS!)
tid_models = {
    'SRAM_TID':[12.2,0.1],
    'MCU_TID':[9.21,0.1],
    'REG_TID':[9.61,0.1],
    'WDT_TID':[11.4,0.1],
}
P = pd.DataFrame(index=env.index)
for effect in effects: P[effect] = 1 - np.exp(-R[effect].mul(pd.Series(R.index).diff().bfill().values).astype(float))
for effect in P.columns: R[effect] = np.absolute(-np.log(1 - P[effect]).div(pd.Series(R.index).diff().bfill().values))
for i in range(len(P)): P.iloc[i] = P.iloc[i] + P.iloc[max(0,i-1)] - P.iloc[i] * P.iloc[max(0,i-1)]
for key in tid_models.keys(): P[key] = models.lognormal(tid,*tid_models[key])
effects += tid_models.keys()
parts = list(set([x.split('_')[0] for x in effects]))
print('effects',effects)
print('parts',parts)

env['Total Dose (rad)'] = env['Total Dose (rad)'].shift(1).fillna(0)
P = P.shift(1).fillna(0)

env.to_csv(f'{project}/tables/env.csv')
display(env)
R.to_csv(f'{project}/tables/rates.csv')
P.to_csv(f'{project}/tables/events.csv')
display(P)
fig = px.line(P,log_y=True,labels={'value':'Probability','variable':'Event'})
fig.update_traces(line={'width':1})
fig.update_layout(
    width = 900,
).write_image(f'{project}/figures/events.png',scale=3)
fig.show()

CREME bins [0.89 1.14 1.46 1.86 2.38 3.04 3.88 4.96 6.34]
L bins [1.015 1.3   1.66  2.12  2.71  3.46  4.42  5.65  7.22 ]
Trajectory L min/max [1.1151, 6.7506]


KeyError: "['REG_SET'] not found in axis"